In [90]:
import os

In [91]:
%pwd

'e:\\'

In [92]:
os.chdir("House-Price-Prediction")

In [93]:
%pwd

'e:\\House-Price-Prediction'

In [95]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [96]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [97]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.data_root])
        
    def get_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config
        
        

In [98]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [100]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def clean_data(self, data):
        # Perform data cleaning operations here
        cleaned_data = data.dropna(subset=['total_bedrooms'])
        return cleaned_data
    
    def handle_categorical_data(self, data):
        # Perform categorical data handling operations here
        processed_data = pd.get_dummies(data, columns=['ocean_proximity'])
        return processed_data

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        
        # data cleaning(drop all rows with missing values)
        cleaned_data = self.clean_data(data)
        
        # Handling categorical data
        processed_data = self.handle_categorical_data(cleaned_data)
        
        # Split the data into training and testing sets, (0.9,0.1) splits.
        train, test = train_test_split(processed_data, test_size=0.1)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("splitted data into training and testing sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        print(train.head())
        
        

In [102]:

try:
    config = ConfigManager()
    data_transformation_config = config.get_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-07-06 11:37:15,582: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-06 11:37:15,585: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-06 11:37:15,602: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-06 11:37:15,603: INFO: common: created directory at: data]
[2024-07-06 11:37:15,604: INFO: common: created directory at: data/data_transformation]
[2024-07-06 11:37:15,762: INFO: 1859079250: splitted data into training and testing sets]
[2024-07-06 11:37:15,762: INFO: 1859079250: (18389, 14)]
[2024-07-06 11:37:15,762: INFO: 1859079250: (2044, 14)]
(18389, 14)
(2044, 14)
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
17971    -121.95     37.32                39.0       1164.0           199.0   
8761     -118.38     33.82                34.0       1822.0           364.0   
12226    -116.92     33.63                18.0        397.0            89.0   
18008    -121.97     37.29                2